# MCP Gateway with LLM Integration

Demonstrates how LLMs use MCP tools to execute code safely.

**What you'll learn:**

- Start HTTP MCP server
- LLM calls tools via MCP
- Workflow orchestration with tool calling

## Step 1: Install & Start MCP Gateway

**First time setup (~2-3 minutes):**

- Installs npm dependencies (30-60s)
- Downloads BGE-M3 model (60-90s) - 2.2GB
- Initializes database & GraphRAG engine

**Subsequent runs:** Server starts in ~5 seconds (model is cached).

In [ ]:
// Start the real MCP Gateway server
console.log("🚀 Starting Casys PML Gateway...");
console.log("");
console.log("📦 First time? This will:");
console.log("   1. Install npm dependencies (~30-60s)");
console.log("   2. Download BGE-M3 model (~60-90s, 2.2GB)");
console.log("   3. Initialize database & GraphRAG");
console.log("");
console.log("⏳ Please wait, this may take 2-3 minutes on first run...");
console.log("");

const server = new Deno.Command("deno", {
  args: ["run", "--allow-all", "../examples/server.ts"],
  stdout: "piped",
  stderr: "piped",
});

const serverProcess = server.spawn();

// Read server output to show progress
const decoder = new TextDecoder();
let ready = false;

// Stream output with timeout
const timeoutMs = 180000; // 3 minutes
const startTime = Date.now();

while (!ready && (Date.now() - startTime) < timeoutMs) {
  const buf = new Uint8Array(1024);
  try {
    const n = await serverProcess.stdout.read(buf);
    if (n) {
      const output = decoder.decode(buf.subarray(0, n));
      console.log(output);
      if (output.includes("✅ MCP Gateway ready!")) {
        ready = true;
      }
    }
  } catch (e) {
    // Continue on read errors
  }
  await new Promise((r) => setTimeout(r, 100));
}

if (!ready) {
  console.log("");
  console.log("⚠️  Server taking longer than expected. Checking health...");
}

// Verify server is responding
await new Promise((r) => setTimeout(r, 2000));
const health = await fetch("http://localhost:3000/health");
const healthData = await health.json();

console.log("");
console.log("✅ MCP Gateway is ready!");
console.log(`   Status: ${healthData.status}`);
console.log(`   Port: 3000`);

🚀 Starting Casys PML Gateway...

📦 First time? This will:
   1. Install npm dependencies (~30-60s)
   2. Download BGE-M3 model (~60-90s, 2.2GB)
   3. Initialize database & GraphRAG

⏳ Please wait, this may take 2-3 minutes on first run...



In [ ]:
// Import visualization helpers
import { dagToMermaid, displayDag } from "../lib/viz.ts";

console.log("📊 DAG visualization loaded (with display functions)");

## Step 2: List Available MCP Tools

Query the MCP server using JSON-RPC 2.0 protocol:

In [ ]:
// MCP uses JSON-RPC 2.0 protocol
const response = await fetch("http://localhost:3000/message", {
  method: "POST",
  headers: { "Content-Type": "application/json" },
  body: JSON.stringify({
    jsonrpc: "2.0",
    id: 1,
    method: "tools/list",
    params: {},
  }),
});

const { result } = await response.json();
const { tools } = result;

console.log(`📋 Available MCP tools (${tools.length}):\n`);
for (const tool of tools) {
  console.log(`🔧 ${tool.name}`);
  console.log(`   ${tool.description.slice(0, 80)}...`);
  console.log();
}

## Step 3: Call Tool Directly

Test tool execution without LLM:

In [ ]:
// Call MCP tool using JSON-RPC
const response = await fetch("http://localhost:3000/message", {
  method: "POST",
  headers: { "Content-Type": "application/json" },
  body: JSON.stringify({
    jsonrpc: "2.0",
    id: 2,
    method: "tools/call",
    params: {
      name: "pml__pml_execute_code",
      arguments: {
        code: "return Array.from({length: 10}, (_, i) => i * i)",
      },
    },
  }),
});

const { result } = await response.json();
console.log("✅ Execution result:");
console.log(JSON.stringify(result, null, 2));

## Step 4: LLM with Tool Calling

Now let the LLM decide when to use tools:

In [ ]:
// Import LLM provider
import { createLLM, generateCompletion } from "../examples/llm-provider.ts";
import { generateText, tool } from "npm:ai";
import { z } from "npm:zod";

const apiKey = Deno.env.get("ANTHROPIC_API_KEY") ||
  Deno.env.get("OPENAI_API_KEY") ||
  Deno.env.get("GOOGLE_API_KEY");

if (!apiKey) {
  throw new Error("Set an API key in llm-demo.ipynb first!");
}

const model = createLLM({ apiKey });

console.log("✅ LLM ready");

Define the tool for the LLM:

In [ ]:
const executeCodeTool = tool({
  description: "Execute TypeScript/JavaScript code safely via MCP",
  parameters: z.object({
    code: z.string().describe("TypeScript/JavaScript code to execute"),
    context: z.any().optional().describe("Optional context data"),
  }),
  execute: async ({ code, context }) => {
    const response = await fetch("http://localhost:3000/message", {
      method: "POST",
      headers: { "Content-Type": "application/json" },
      body: JSON.stringify({
        jsonrpc: "2.0",
        id: Math.random(),
        method: "tools/call",
        params: {
          name: "pml__pml_execute_code",
          arguments: { code, context },
        },
      }),
    });
    const { result } = await response.json();
    return result.content[0].text;
  },
});

## Step 5: Ask LLM to Solve Task with Code

The LLM will generate and execute code:

In [ ]:
const result = await generateText({
  model,
  tools: { execute_code: executeCodeTool },
  maxSteps: 5,
  prompt:
    "Calculate the sum of all prime numbers between 1 and 100. Write and execute TypeScript code to do this.",
});

console.log("\n🤖 LLM Response:");
console.log(result.text);

console.log("\n📊 Tool Calls:");
for (const step of result.steps) {
  if (step.toolCalls) {
    for (const call of step.toolCalls) {
      console.log(`\n🔧 Called: ${call.toolName}`);
      console.log("   Args:", JSON.stringify(call.args, null, 2));
      console.log("   Result:", JSON.stringify(step.toolResults, null, 2));
    }
  }
}

## Step 6: Multi-Step Workflow

LLM orchestrates multiple tool calls:

In [ ]:
// The workflow above could be represented as a DAG
const workflowDAG = {
  id: "sales-analysis",
  tasks: [
    { id: "parse_data", tool_name: "execute_code", dependencies: [] },
    { id: "calc_revenue", tool_name: "execute_code", dependencies: ["parse_data"] },
    { id: "find_best_seller", tool_name: "execute_code", dependencies: ["parse_data"] },
    { id: "calc_avg_price", tool_name: "execute_code", dependencies: ["parse_data"] },
    {
      id: "generate_report",
      tool_name: "execute_code",
      dependencies: ["calc_revenue", "find_best_seller", "calc_avg_price"],
    },
  ],
};

console.log("📊 Sales Analysis Workflow as DAG:");
console.log("💡 Steps 2-4 could run in parallel!\n");

// Display visual diagram
await displayDag(workflowDAG);

## Visualize Workflow as DAG

The multi-step workflow can be represented as a DAG:

In [ ]:
const workflow = await generateText({
  model,
  tools: { execute_code: executeCodeTool },
  maxSteps: 10,
  prompt: `Analyze this sales data step by step:
1. Calculate total revenue
2. Find best selling product
3. Calculate average price

Data: [{"product":"Laptop","price":1200,"qty":2},{"product":"Mouse","price":25,"qty":5},{"product":"Keyboard","price":80,"qty":3}]

Write code to perform each calculation.`,
});

console.log("\n🤖 Workflow Result:");
console.log(workflow.text);

console.log("\n📈 Steps executed:", workflow.steps.length);

## Step 7: Cleanup

Stop the MCP server:

In [ ]:
serverProcess.kill("SIGTERM");
await serverProcess.status;
console.log("✅ Server stopped");

## Summary

**What we demonstrated:**

- ✅ MCP server exposes tools via HTTP
- ✅ LLM discovers and calls tools automatically
- ✅ Safe code execution in sandbox
- ✅ Multi-step workflow orchestration
- ✅ LLM decides when and how to use tools

**The MCP Pattern:**

```
User Query → LLM → Tool Selection → MCP Server → Sandbox → Results → LLM → User
```

This is how Claude Code, Cline, and other AI coding tools work!